In [1]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 8.1 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
import gradio as gr
import random
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth


In [3]:
import os

# Replace 'YOUR_CLIENT_ID' and 'YOUR_CLIENT_SECRET' with your actual credentials
os.environ['SPOTIPY_CLIENT_ID'] = '5a49208c06b4449abbaa765c9e896d03'
os.environ['SPOTIPY_CLIENT_SECRET'] = '97a8aac732c7450e8e9bde4af134729a'
os.environ['SPOTIPY_REDIRECT_URI'] = 'https://mywebsite.com/callback'
# Spotify Authentication
scope = "playlist-read-private playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=os.environ['SPOTIPY_CLIENT_ID'],
    client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],
    redirect_uri=os.environ['SPOTIPY_REDIRECT_URI'],
    scope=scope
))


In [4]:
# Get music from Spotify based on emotion
def get_music_by_emotion(emotion):
    try:
        query = f"{emotion} music"
        results = sp.search(q=query, type='track', limit=10)
        tracks_info = []
        for track in results['tracks']['items']:
            tracks_info.append({
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'album': track['album']['name'],
                'uri': track['uri']
            })
        return tracks_info
    except Exception as e:
        print(f"Error fetching music: {e}")
        return []

# Refine recommendations based on detected emotions
def refine_recommendations(emotion_data, spotify_results):
    if not emotion_data:
        return spotify_results
    # Pick highest confidence emotion
    primary_emotion = max(emotion_data, key=lambda x: x['confidence'])['emotion']
    print(f"Primary emotion: {primary_emotion}")
    return spotify_results


In [5]:
def recommend_music_system(emotion_data):
    if not emotion_data:
        print("No emotion data provided.")
        return []
    primary_emotion = max(emotion_data, key=lambda x: x['confidence'])['emotion']
    initial_recommendations = get_music_by_emotion(primary_emotion)
    final_recommendations = refine_recommendations(emotion_data, initial_recommendations)
    return final_recommendations


In [6]:
# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def get_emotion_from_face(face_roi):
    # Placeholder: random emotion
    emotions = ['happy', 'sad', 'neutral', 'angry', 'surprise']
    return random.choice(emotions), random.uniform(0.4, 0.9)

def detect_emotion_from_video(video_input_path):
    if video_input_path is None:
        return []

    cap = cv2.VideoCapture(video_input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video source {video_input_path}")
        return []

    detected_emotions = []
    frame_count = 0
    process_interval = 5  # Process every 5th frame

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        if frame_count % process_interval != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            emotion, confidence = get_emotion_from_face(face_roi)
            detected_emotions.append({'emotion': emotion, 'confidence': confidence, 'source': 'video'})

    cap.release()
    cv2.destroyAllWindows()

    if not detected_emotions:
        return []

    # Aggregate most common emotion
    emotion_counts = {}
    emotion_confidences = {}
    for entry in detected_emotions:
        e = entry['emotion']
        c = entry['confidence']
        emotion_counts[e] = emotion_counts.get(e, 0) + 1
        emotion_confidences[e] = emotion_confidences.get(e, []) + [c]

    most_common_emotion = max(emotion_counts, key=emotion_counts.get)
    avg_confidence = float(np.mean(emotion_confidences[most_common_emotion]))
    return [{'emotion': most_common_emotion, 'confidence': avg_confidence, 'source': 'video'}]


In [7]:
def recommend_music_interface(video_input):
    emotion_data = detect_emotion_from_video(video_input)
    if not emotion_data:
        return "Could not detect emotion. Try a different video."
    recommended_music = recommend_music_system(emotion_data)
    if recommended_music:
        output_text = "Recommended Music:\n"
        for track in recommended_music:
            output_text += f"- {track['name']} by {track['artist']} (Album: {track['album']})\n"
    else:
        output_text = "Could not find music for the detected emotion."
    return output_text

# Create Gradio interface
iface = gr.Interface(
    fn=recommend_music_interface,
    inputs=gr.Video(label="Video Input"),
    outputs=gr.Textbox(label="Recommended Music"),
    title="Emotion-Based Music Recommender"
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88baa2483323906d84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
